In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls "/content/drive/MyDrive/tmas"

AnnotatedLabel	config	CRYPTIC  Labels


In [ ]:
#testt

In [ ]:
import re
import os
import pandas as pd
from pathlib import Path
import json
import numpy as np

In [ ]:
data_dir = Path("/content/drive/My Drive/tmas")
plate_images_folder =  data_dir / "CRYPTIC" / "plate-images-20240423"
plates_csv_file = data_dir / "CRYPTIC" / "PLATES-20240423.csv"
ukmyc_csv_file = data_dir / "CRYPTIC" / "UKMYC_PHENOTYPES-20240423.csv"
plate_design_file = data_dir / "config" / "plate-design.json"

In [ ]:
def extract_image_name(file_name):
    # Use a regular expression to capture the part of the file name before '-UKMYC[5-6]-filtered.png'
    match = re.search(r"^(.+)-UKMYC[56]-filtered\.png$", file_name)
    if match:
        return match.group(1)
    return None

def read_filenames_and_extract(folder_path):
    # Dictionary to hold image names classified by subfolder
    image_names = {}

    # Loop through the main folders (e.g., "UKMYC5", "UKMYC6")
    for main_folder in os.listdir(folder_path):
        main_folder_path = os.path.join(folder_path, main_folder)
        if os.path.isdir(main_folder_path):  # check if it is a directory
            image_names[main_folder] = {}

            # Loop through each subfolder (e.g., "14", "21")
            for subfolder in os.listdir(main_folder_path):
                subfolder_path = os.path.join(main_folder_path, subfolder)
                if os.path.isdir(subfolder_path):
                    image_names[main_folder][subfolder] = []

                    # List all files in the subfolder
                    for filename in os.listdir(subfolder_path):
                        if filename.endswith('-filtered.png'):  # check if the file is an image
                            extracted_name = extract_image_name(filename)
                            if extracted_name:
                                image_names[main_folder][subfolder].append(extracted_name)

    return image_names

In [ ]:
image_file_names = read_filenames_and_extract(plate_images_folder)
print(image_file_names)

{'UKMYC5': {'21': ['04-00050-633275-1-21', '04-00552-715649-1-21', '03-UA11-2017-UA11-2017-1-21', '04-00811-707860-1-21', '04-00878-709531-1-21', '04-00859-709463-1-21', '04-00031-629762-1-21', '04-00751-709109-1-21', '04-00183-700782-1-21', '04-00820-707874-1-21', '04-00522-714514-1-21', '04-00819-706792-1-21', '04-00821-708090-1-21', '04-00829-706270-1-21', '04-00946-717248-1-21', '04-00815-708446-1-21', '04-00824-708449-1-21', '04-00123-630236-1-21', '04-00840-707921-1-21', '04-00583-T-4170-1-21', '02-0932-22A167-1-21', '03-JPN-R2012-00013-03-JPN-R2012-00013-1-21', '04-00822-707857-1-21', '04-00884-714267-1-21', '04-00246-701573-1-21', '03-BOR-1304-17-BOR-1304-17-1-21', '04-00245-702669-1-21', '04-00151-634475-1-21', '03-BOR-3145-17-3145-17-1-21', '04-00163-631936-1-21', '04-00363-700271-1-21', '03-BOR-6874-17-6874-17-1-21', '04-00789-707540-1-21', '04-00388-705145-1-21', '04-00052-633952-1-21', '04-00846-709555-1-21', '06-06TB_0032-06MIL1277-1-21', '06-06TB_0290-06MIL0881-1-21', '0

In [ ]:
print(len(image_file_names['UKMYC5']['14']))
print(len(image_file_names['UKMYC5']['21']))
print(len(image_file_names['UKMYC6']['14']))
print(len(image_file_names['UKMYC6']['21']))

5731
135
8888
384


In [ ]:
image_names = []

image_names.extend(image_file_names['UKMYC5']['14'])
image_names.extend(image_file_names['UKMYC5']['21'])
image_names.extend(image_file_names['UKMYC6']['14'])
image_names.extend(image_file_names['UKMYC6']['21'])

print(f"Total number of images: {len(image_names)}")

Total number of images: 15138


In [ ]:
def load_plate_csv_data(csv_path):
    # Load only the necessary columns for efficiency
    data = pd.read_csv(csv_path, usecols=['UNIQUEID', 'IMAGEFILENAME'])

    # Create a dictionary to map image filenames to unique IDs
    image_to_unique_id = dict(zip(data['IMAGEFILENAME'], data['UNIQUEID']))

    return image_to_unique_id

def find_unique_ids(image_file_names, image_to_unique_id):
    # Dictionary to store unique IDs matched to image file names
    unique_ids = {}

    # Loop through each image file name to find corresponding unique IDs
    for image_file_name in image_file_names:
        if image_file_name in image_to_unique_id:
            unique_ids[image_file_name] = image_to_unique_id[image_file_name]
        else:
            unique_ids[image_file_name] = None  # no match found

    return unique_ids

In [ ]:
image_to_unique_id = load_plate_csv_data(plates_csv_file) # load data from "PLATES-20240423.csv"
unique_ids = find_unique_ids(image_names, image_to_unique_id)

print(len(unique_ids))
print(unique_ids)

15138
{'08-05TB42023-1830-1-14': 'site.08.subj.05TB42023.lab.1830.iso.1', '08-05TB43017-1894-1-14': 'site.08.subj.05TB43017.lab.1894.iso.1', '08-24TB-004-2088-1-14': 'site.08.subj.24TB-004.lab.2088.iso.1', '08-24TB-074-2208-1-14': 'site.08.subj.24TB-074.lab.2208.iso.1', '08-05TB43024-1950-1-14': 'site.08.subj.05TB43024.lab.1950.iso.1', '08-05TB42053-1937-1-14': 'site.08.subj.05TB42053.lab.1937.iso.1', '08-24TB-081-2249-1-14': 'site.08.subj.24TB-081.lab.2249.iso.1', '08-05TB42007-1771-1-14': 'site.08.subj.05TB42007.lab.1771.iso.1', '08-05TB41041-1941-1-14': 'site.08.subj.05TB41041.lab.1941.iso.1', '08-05TB43007-1818-1-14': 'site.08.subj.05TB43007.lab.1818.iso.1', '08-24TB00-018-2313-1-14': 'site.08.subj.24TB00-018.lab.2313.iso.1', '08-24TB00-005-2246-1-14': 'site.08.subj.24TB00-005.lab.2246.iso.1', '08-05TB42038-1871-1-14': 'site.08.subj.05TB42038.lab.1871.iso.1', '08-24TB00-043-2438-1-14': 'site.08.subj.24TB00-043.lab.2438.iso.1', '08-24TB00-063-2401-1-14': 'site.08.subj.24TB00-063.lab

In [ ]:
ukmyc_dataset = pd.read_csv(ukmyc_csv_file, usecols=['UNIQUEID', 'DRUG', 'PLATEDESIGN', 'MIC'])
print(ukmyc_dataset.head())

                                       UNIQUEID DRUG PLATEDESIGN      MIC
0      site.05.subj.PMFR-0743.lab.MFR-247.iso.1  DLM      UKMYC5  <=0.015
1  site.10.subj.YA00033005.lab.YA00033005.iso.1  CFZ      UKMYC5     0.25
2    site.06.subj.06TB_1032.lab.06MIL2037.iso.1  MXF      UKMYC6     0.25
3        site.02.subj.1892.lab.2015184043.iso.1  INH      UKMYC6  <=0.025
4    site.06.subj.06TB_0404.lab.06MIL1373.iso.1  MXF      UKMYC6     0.25


In [ ]:
# Sort the DataFrame based on the 'UNIQUEID' column
ukmyc_dataset_sorted = ukmyc_dataset.sort_values(by='UNIQUEID')
print(ukmyc_dataset_sorted.head(13))

                                    UNIQUEID DRUG PLATEDESIGN    MIC
187074  site.01.subj.DR0013.lab.DR0013.iso.1  CFZ      UKMYC6   0.06
115804  site.01.subj.DR0013.lab.DR0013.iso.1  BDQ      UKMYC6   0.06
160827  site.01.subj.DR0013.lab.DR0013.iso.1  EMB      UKMYC6    4.0
156376  site.01.subj.DR0013.lab.DR0013.iso.1  LEV      UKMYC6    0.5
29369   site.01.subj.DR0013.lab.DR0013.iso.1  KAN      UKMYC6    4.0
25885   site.01.subj.DR0013.lab.DR0013.iso.1  DLM      UKMYC6  0.015
108568  site.01.subj.DR0013.lab.DR0013.iso.1  LZD      UKMYC6    1.0
72112   site.01.subj.DR0013.lab.DR0013.iso.1  MXF      UKMYC6   0.25
17841   site.01.subj.DR0013.lab.DR0013.iso.1  AMI      UKMYC6    0.5
159600  site.01.subj.DR0013.lab.DR0013.iso.1  RIF      UKMYC6    8.0
149449  site.01.subj.DR0013.lab.DR0013.iso.1  ETH      UKMYC6    1.0
47865   site.01.subj.DR0013.lab.DR0013.iso.1  RFB      UKMYC6    2.0
10516   site.01.subj.DR0013.lab.DR0013.iso.1  INH      UKMYC6    1.6


In [ ]:
def find_image_name_by_unique_id(unique_id, unique_ids_dict):
    for image_name, id in unique_ids_dict.items():
        if id == unique_id:
            return image_name
    return None

def find_unique_id_by_image_name(image_name, unique_ids_dict):
    return unique_ids_dict.get(image_name, None)

In [ ]:
search_unique_id = "site.01.subj.DR0013.lab.DR0013.iso.1"
print(find_image_name_by_unique_id(search_unique_id, unique_ids))

01-DR0013-DR0013-1-14


In [ ]:
search_image_name = "01-DR0013-DR0013-1-14"
print(find_unique_id_by_image_name(search_image_name, unique_ids))

site.01.subj.DR0013.lab.DR0013.iso.1


In [ ]:
with open(plate_design_file, 'r') as file:
    plate_design = json.load(file)

print(plate_design)

{'UKMYC5': {'drug_matrix': [['BDQ', 'KAN', 'KAN', 'KAN', 'KAN', 'KAN', 'ETH', 'ETH', 'ETH', 'ETH', 'ETH', 'ETH'], ['BDQ', 'AMI', 'EMB', 'INH', 'LEV', 'MXF', 'DLM', 'LZD', 'CFZ', 'RIF', 'RFB', 'PAS'], ['BDQ', 'AMI', 'EMB', 'INH', 'LEV', 'MXF', 'DLM', 'LZD', 'CFZ', 'RIF', 'RFB', 'PAS'], ['BDQ', 'AMI', 'EMB', 'INH', 'LEV', 'MXF', 'DLM', 'LZD', 'CFZ', 'RIF', 'RFB', 'PAS'], ['BDQ', 'AMI', 'EMB', 'INH', 'LEV', 'MXF', 'DLM', 'LZD', 'CFZ', 'RIF', 'RFB', 'PAS'], ['BDQ', 'AMI', 'EMB', 'INH', 'LEV', 'MXF', 'DLM', 'LZD', 'CFZ', 'RIF', 'RFB', 'PAS'], ['BDQ', 'AMI', 'EMB', 'INH', 'LEV', 'MXF', 'DLM', 'LZD', 'CFZ', 'RIF', 'RFB', 'PAS'], ['BDQ', 'EMB', 'EMB', 'INH', 'LEV', 'MXF', 'DLM', 'LZD', 'CFZ', 'RIF', 'POS', 'POS']], 'dilution_matrix': [[8, 5, 4, 3, 2, 1, 6, 5, 4, 3, 2, 1], [7, 6, 8, 7, 7, 7, 7, 7, 7, 7, 6, 6], [6, 5, 7, 6, 6, 6, 6, 6, 6, 6, 5, 5], [5, 4, 6, 5, 5, 5, 5, 5, 5, 5, 4, 4], [4, 3, 5, 4, 4, 4, 4, 4, 4, 4, 3, 3], [3, 2, 4, 3, 3, 3, 3, 3, 3, 3, 2, 2], [2, 1, 3, 2, 2, 2, 2, 2, 2, 2, 1, 1

In [ ]:
def apply_mic_rules(mic):
    # Check if mic is a string and starts with special characters
    if isinstance(mic, str):
        if mic.startswith('<='):
            return '-none-', float(mic[2:])
        elif mic.startswith('>'):
            return 'growth', float(mic[1:])
    # Handle cases where mic is already a float or can be converted to float
    try:
        return 'numerical', float(mic)
    except ValueError:
        raise ValueError(f"MIC value {mic} is not properly formatted or is not a numeric value.")


def mark_wells(plate_design, drug_data):
    marked_plates = {}  # Dictionary to store all marked plates by unique ID

    # Process each unique ID
    for unique_id in drug_data['UNIQUEID'].unique():
        specific_data = drug_data[drug_data['UNIQUEID'] == unique_id]
        target_design = specific_data['PLATEDESIGN'].iloc[0]

        if target_design not in plate_design:
            print(f"No data available for plate design {target_design}")
            continue

        # Initialize the marked plate
        marked_plate = [['untested' for _ in range(12)] for _ in range(8)]
        details = plate_design[target_design]
        drug_matrix = details['drug_matrix']
        dilution_matrix = details['dilution_matrix']
        conc_matrix = details['conc_matrix']

        for i in range(len(drug_matrix)):
            for j in range(len(drug_matrix[i])):
                drug = drug_matrix[i][j]
                mic_entry = specific_data[(specific_data['DRUG'] == drug) & (specific_data['PLATEDESIGN'] == target_design)]

                if not mic_entry.empty:
                    mic_type, mic_value = apply_mic_rules(mic_entry['MIC'].values[0])

                    if mic_type == 'growth':
                        status = 'growth'
                    elif mic_type == '-none-':
                        status = '-none-'
                    else:  # Numerical MIC
                        conc = conc_matrix[i][j]
                        status = 'growth' if conc <= mic_value else '-none-'
                else:
                    if drug == "POS":
                        status = 'growth'
                    else:
                        status = 'untested'

                marked_plate[i][j] = status

        marked_plates[unique_id] = marked_plate  # Store the marked plate with its unique ID

    return marked_plates

# Plates to be marked - dictionary
marked_plates = mark_wells(plate_design, ukmyc_dataset_sorted)

# Print out the marked plates for each unique ID
# if marked_plates:
#     for unique_id, plate in marked_plates.items():
#         print(f"Plate Unique ID: {unique_id}")
#         for row in plate:
#             print(' '.join(row))
#         print("\n")


In [ ]:
# marked_plates_example = marked_plates.head(3)
# print(marked_plates_example)

In [ ]:
print(marked_plates['site.02.subj.0004.lab.2014222010.iso.1'])

[['-none-', '-none-', '-none-', '-none-', 'growth', 'growth', '-none-', 'growth', 'growth', 'growth', 'growth', 'growth'], ['-none-', '-none-', '-none-', '-none-', '-none-', '-none-', '-none-', '-none-', '-none-', '-none-', '-none-', '-none-'], ['-none-', '-none-', '-none-', '-none-', '-none-', '-none-', '-none-', '-none-', '-none-', '-none-', '-none-', '-none-'], ['-none-', '-none-', 'growth', '-none-', '-none-', '-none-', '-none-', 'growth', '-none-', '-none-', '-none-', '-none-'], ['-none-', '-none-', 'growth', '-none-', 'growth', '-none-', '-none-', 'growth', '-none-', '-none-', '-none-', '-none-'], ['growth', '-none-', 'growth', '-none-', 'growth', 'growth', '-none-', 'growth', '-none-', '-none-', '-none-', '-none-'], ['growth', '-none-', 'growth', 'growth', 'growth', 'growth', '-none-', 'growth', 'growth', 'growth', '-none-', '-none-'], ['growth', 'growth', 'growth', 'growth', 'growth', 'growth', '-none-', 'growth', 'growth', 'growth', 'growth', 'growth']]


In [ ]:
# Get image name based on id faster
reversed_unique_ids = {v: k for k, v in unique_ids.items()}

In [ ]:
import cv2
from pathlib import Path

def get_plate_design(unique_id, ukmyc_dataset_sorted):
    """ Retrieve the plate design for a given unique ID from the sorted DataFrame. """
    filtered = ukmyc_dataset_sorted[ukmyc_dataset_sorted['UNIQUEID'] == unique_id]
    return filtered.iloc[0]['PLATEDESIGN'] if not filtered.empty else None

def load_image(image_path):
    """ Load an image from the given path. """
    image = cv2.imread(str(image_path))
    if image is None:
        raise FileNotFoundError(f"Failed to load image: {image_path}")
    return image

def draw_growth_circles(image, marked_plate, well_center, well_radius):
    """ Draw circles on growth positions with adjusted radius. """
    num_rows, num_cols = len(marked_plate), len(marked_plate[0])
    for i in range(num_rows):
        for j in range(num_cols):
            if marked_plate[i][j] == 'growth':
                center = (int(well_center[i, j][0]), int(well_center[i, j][1]))
                adjusted_radius = int(well_radius[i, j] * 0.72)  # Reducing radius by 30%
                cv2.circle(image, center, adjusted_radius, (0, 255, 0), 2)


def identify_wells(image,hough_param1=20,hough_param2=25,radius_tolerance=0.005,verbose=False):
        well_dimensions = (8, 12)


        well_index = np.zeros(well_dimensions, dtype=int)
        well_radii = np.zeros(well_dimensions, dtype=float)
        well_center = np.zeros((well_dimensions[0], well_dimensions[1], 2), dtype=int)
        well_top_left = np.zeros((well_dimensions[0], well_dimensions[1], 2), dtype=int)
        well_bottom_right = np.zeros((well_dimensions[0], well_dimensions[1], 2), dtype=int)


        # Number of wells
        number_of_wells = well_dimensions[0] * well_dimensions[1]

        # # Read in the image in grayscale
        # image = cv2.imread(image_path)
        # if image is None:
        #     raise ValueError("Image could not be loaded, check the path.")

        # Image dimensions
        image_dimensions = image.shape  # Assumes image is already in grayscale

        # estimate the dimensions of the well
        estimate_well_y = float(image_dimensions[0])/well_dimensions[0]
        estimate_well_x = float(image_dimensions[1])/well_dimensions[1]

        # verify that the estimated dimensions of the wells are within 5% of one another
        if estimate_well_x > estimate_well_y:
            if estimate_well_x > 1.05*estimate_well_y:
                return False
        else:
            if estimate_well_y > 1.05*estimate_well_x:
                return False

        # now estimate the radius as the mean of half the estimated well dimension
        estimated_radius=(estimate_well_x + estimate_well_y)/4.

        radius_multiplier=1.+radius_tolerance

        # read in a greyscale image for identifying
        grey_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # iterate until the correct number of circles have been found
        while True:

            circles=None

            while circles is None:

                # detect circles using the Hough transform
                circles=cv2.HoughCircles(grey_image,cv2.HOUGH_GRADIENT,1,50,param1=hough_param1,param2=hough_param2,minRadius=int(estimated_radius/radius_multiplier),maxRadius=int(estimated_radius*radius_multiplier))

                # increase the range of radii that will be searched for
                radius_multiplier+=radius_tolerance

                # if verbose:
                    # print("No circles, "+str(int(estimated_radius/radius_multiplier))+" < radius < "+str(int(estimated_radius*radius_multiplier)))

            # count how many circles there are
            number_of_circles=len(circles[0])

            if verbose:
                print(str(number_of_circles)+" circles, "+str(int(estimated_radius/radius_multiplier))+" < radius < "+str(int(estimated_radius*radius_multiplier)))

            # check to see if there are enough circles
            if number_of_circles>=number_of_wells:
                break
            elif number_of_circles>number_of_wells:
                # print(self.image_path+": "+str(number_of_circles)+" circles found, this is more than the expected number of "+str(self.number_of_wells))
                break
            elif radius_multiplier>2:
                # print(self.image_path+": "+"reached maximum radius multiplier of 2 and found "+str(number_of_circles)+" circles, giving up")
                break
            else:
                radius_multiplier+=radius_tolerance

        well_counter=0

        one_circle_per_well=True

        # move along the wells in the x-direction (columns)
        for ix in range(0,well_dimensions[1]):

            # move along the wells in the y-direction (rows)
            for iy in range(0,well_dimensions[0]):

                # calculate the bottom left and top right coordinates of the well
                top_left=(int(ix*estimate_well_x), int(iy*estimate_well_y))
                bottom_right=(int((ix+1)*estimate_well_x), int((iy+1)*estimate_well_y))

                number_of_circles_in_well=0

                # loop over the list of identified wells
                for ic in circles[0,]:

                    # check to see if the circle lies with the well
                    if top_left[0] < ic[0] < bottom_right[0]:
                        if top_left[1] < ic[1] < bottom_right[1]:
                            number_of_circles_in_well+=1
                            circle=ic

                if number_of_circles_in_well==1:

                    well_centre=(circle[0],circle[1])
                    well_radius=circle[2]
                    well_extent=1.2*well_radius

                    x1=max(0,int(well_centre[0]-well_extent))
                    x2=min(image_dimensions[1],int(well_centre[0]+well_extent))

                    y1=max(0,int(well_centre[1]-well_extent))
                    y2=min(image_dimensions[0],int(well_centre[1]+well_extent))

                    well_index[iy,ix]= well_counter
                    well_center[iy,ix] = well_centre
                    well_radii[iy,ix] = well_radius
                    well_top_left[iy,ix] = (max(0,int(well_centre[0]-well_extent)),max(0,int(well_centre[1]-well_extent)))
                    well_bottom_right[iy,ix] = (min(image_dimensions[1],int(well_centre[0]+well_extent)),min(image_dimensions[0],int(well_centre[1]+well_extent)))
                    well_counter+=1
                else:
                    if verbose:
                        print(number_of_circles_in_well)
                    one_circle_per_well=False



        if well_counter==number_of_wells and one_circle_per_well:
            return well_center, well_radii
        else:
            return False


def annotate_and_save_images(reversed_unique_ids, unique_ids, marked_plates, base_dir, ukmyc_dataset_sorted, file_extension='png'):
    base_dir = Path(base_dir)
    output_dir = base_dir / 'Labels'
    output_dir.mkdir(exist_ok=True)

    for detailed_id in unique_ids:
        simplified_id = reversed_unique_ids.get(detailed_id)
        if not simplified_id:
            print(f"No image identifier found for unique ID: {detailed_id}")
            continue

        plate_design = get_plate_design(detailed_id, ukmyc_dataset_sorted)
        if not plate_design:
            print(f"No plate design data available for unique ID: {detailed_id}")
            continue

        subfolder = '14' if '14' in simplified_id else '21'
        image_dir = base_dir / "CRYPTIC" / "plate-images-20240423" / plate_design / subfolder
        image_path = image_dir / f"{simplified_id}-{plate_design}-filtered.{file_extension}"

        try:
            image = load_image(image_path)
            result = identify_wells(image, verbose=True)
            if not result:
                print(f"Failed to identify wells properly in image: {image_path}")
                continue

            well_center, well_radii = result
            draw_growth_circles(image, marked_plates[detailed_id], well_center, well_radii)
            output_path = output_dir / f"{simplified_id}_annotated.{file_extension}"
            cv2.imwrite(str(output_path), image)
            print(f"Annotated image saved: {output_path}")
        except FileNotFoundError as e:
            print(e)
        except Exception as e:
            print(f"An error occurred while processing image: {image_path}")
            print(str(e))

In [ ]:
# # Sample to annotate just one image

# base_directory = "/content/drive/My Drive/tmas"
# unique_ids_to_process = ['site.02.subj.0004.lab.2014222010.iso.1']
# annotate_and_save_images(reversed_unique_ids, unique_ids_to_process, marked_plates, base_directory, ukmyc_dataset_sorted)

In [ ]:
base_directory = "/content/drive/MyDrive/tmas"
# Get all unique IDs from the keys of the reversed_unique_ids dictionary

unique_ids_to_process = list(reversed_unique_ids.keys())

In [ ]:
# Call the function to process all these IDs
annotate_and_save_images(reversed_unique_ids, unique_ids_to_process, marked_plates, base_directory, ukmyc_dataset_sorted)

Streaming output truncated to the last 5000 lines.
94 circles, 38 < radius < 43
94 circles, 38 < radius < 43
96 circles, 37 < radius < 44
Annotated image saved: /content/drive/MyDrive/tmas/Labels/02-1545-2013102383-1-14_annotated.png
20 circles, 39 < radius < 42
49 circles, 39 < radius < 42
49 circles, 38 < radius < 42
94 circles, 38 < radius < 43
94 circles, 38 < radius < 43
96 circles, 37 < radius < 44
Annotated image saved: /content/drive/MyDrive/tmas/Labels/02-1506-2013101204-1-14_annotated.png
2 circles, 40 < radius < 41
2 circles, 40 < radius < 41
7 circles, 39 < radius < 42
39 circles, 39 < radius < 42
39 circles, 39 < radius < 43
84 circles, 38 < radius < 43
94 circles, 38 < radius < 44
95 circles, 38 < radius < 44
96 circles, 37 < radius < 44
Annotated image saved: /content/drive/MyDrive/tmas/Labels/02-1716-2015185033-1-14_annotated.png
2 circles, 40 < radius < 41
2 circles, 40 < radius < 41
18 circles, 39 < radius < 42
72 circles, 39 < radius < 42
72 circles, 38 < radius < 42

In [ ]:
def count_files_in_directory(directory_path):
    """ Count the number of files in the specified directory. """
    try:
        # List all entries in the directory
        entries = os.listdir(directory_path)
        # Filter out directories and count only files
        file_count = sum(1 for entry in entries if os.path.isfile(os.path.join(directory_path, entry)))
        return file_count
    except FileNotFoundError:
        return "The specified directory does not exist."
    except PermissionError:
        return "Permission denied accessing the directory."
    except Exception as e:
        return f"An error occurred: {e}"

# Specify the directory path
directory_path = '/content/drive/MyDrive/tmas/Labels'  # Modify this path to the correct location on your filesystem
# Call the function and print the result
print(f"Number of files in '{directory_path}': {count_files_in_directory(directory_path)}")
print(f"Number of files in '{'/content/drive/MyDrive/tmas/Labels/NotAnnotate'}': {count_files_in_directory('/content/drive/MyDrive/tmas/Labels/NotAnnotate')}")

Number of files in '/content/drive/MyDrive/tmas/Labels': 15009
Number of files in '/content/drive/MyDrive/tmas/Labels/NotAnnotate': 127


In [ ]:
def find_missing_images(directory_path, reversed_unique_ids, unique_ids_to_process):
    # Prepare a set of existing files for quick lookup
    existing_files = set(os.listdir(directory_path))

    # Dictionary to store the result for missing files only
    missing_results = {}

    # Loop over the unique IDs to process
    for unique_id in unique_ids_to_process:
        # Retrieve the identifier used to create filenames from the dictionary
        identifier = reversed_unique_ids.get(unique_id, None)
        if identifier:
            # Format the expected filename based on the unique identifier
            expected_filename = f"{identifier}_annotated.png"

            # Check if the file does NOT exist in the directory
            if expected_filename not in existing_files:
                missing_results[unique_id] = "File does not exist"
        else:
            # Handle cases where the identifier itself is missing
            missing_results[unique_id] = "Missing identifier"

    return missing_results

# Example usage
base_directory = "/content/drive/MyDrive/tmas"
labels_directory = Path(base_directory) / "Labels"

# Assuming `reversed_unique_ids` is a dictionary loaded elsewhere and properly populated
# unique_ids_to_process = list(reversed_unique_ids.keys())[10550:] # Modify this as necessary

# Find missing images
missing_images = find_missing_images(labels_directory, reversed_unique_ids, unique_ids_to_process)

# Print results for missing files only
print(f"Number of missing files: {len(missing_images)}")
for uid, status in missing_images.items():
    # print(f"Unique ID: {uid}, Status: {status}")
    image_id = reversed_unique_ids.get(uid, None)
    print(image_id)

Number of missing files: 127
08-05TB12001-21742-1-14
04-01059-717554-1-14
04-01106-719160-1-14
04-01075-719392-1-14
04-00522-714514-1-21
04-00884-714267-1-21
03-BOR-3145-17-3145-17-1-21
04-00388-705145-1-21
08-02TB2414-26096-1-21
06-06TB_0463-06MIL1141-1-21
06-MHL_0151-14-06MIL0847-1-21
14-4247-4247-1-21
06-JHL_0022-14-06MIL0004-1-21
10-YA00014253-YA00014253-1-21
08-23TB03-314-2238-1-21
06-06TB_0014-06MIL0475-1-21
06-BVH_0100-14-06MIL0235-1-21
14-4221-4221-1-21
08-05TB22112-27614-1-21
14-4203-4203-1-21
06-PIMS_0043-14-06MIL0302-1-21
06-SSM_0165-14-06MIL0404-1-21
06-PIMS_0020-14-06MIL0109-1-21
06-SGD_0028-14-06MIL0142-1-21
14-4214-4214-1-21
14-4245-4245-1-21
06-SGD_0046-14-06MIL0850-1-21
14-3800-3800-1-21
06-RLH_0053-14-06MIL0013-1-21
14-4249-4249-1-21
10-YA00060146-YA00060146-1-21
10-ATCC191218-ATCC191218-1-14
05-LS-1085-MA-08263-18-1-14
05-LR-2102-FN-01464-15-1-21
03-GB-84140056-GB-84140056-1-21
03-IML-00141-T8147-1-21
05-LR-2100-FN-01461-15-1-21
05-LR-2098-FN-01434-15-1-21
02-1454-20

In [ ]:
# Duplicate those missing images

import os
from pathlib import Path
import shutil

def copy_missing_images_and_collect_ids(base_dir, missing_images, reversed_unique_ids, file_extension='png'):
    labels_dir = base_dir / 'Labels'
    not_annotate_dir = labels_dir / 'NotAnnotate'
    not_annotate_dir.mkdir(exist_ok=True)  # Ensure the NotAnnotate directory exists

    # List to store image names of missing files
    missing_image_names = []

    # Possible configurations of folders
    plate_designs = ['UKMYC5', 'UKMYC6']
    subfolders = ['14', '21']

    # Iterate through the missing_images dictionary
    for uid, status in missing_images.items():
        if status == "File does not exist":
            image_name = reversed_unique_ids.get(uid, None)
            if image_name:
                missing_image_names.append(image_name)  # Add the image_name to the list
                file_found = False

                # Try all possible folder combinations
                for plate_design in plate_designs:
                    for subfolder in subfolders:
                        # Constructing filename using the provided pattern
                        file_name = f"{image_name}-{plate_design}-filtered.{file_extension}"
                        original_image_path = base_dir / "CRYPTIC" / "plate-images-20240423" / plate_design / subfolder / file_name

                        if original_image_path.exists():
                            shutil.copy(original_image_path, not_annotate_dir / original_image_path.name)
                            print(f"Copied missing image to NotAnnotate: {original_image_path.name}")
                            file_found = True
                            break
                    if file_found:
                        break

                if not file_found:
                    print(f"Original image does not exist in any expected directory: {image_name}")

            else:
                print(f"Missing image name for UID: {uid}")

    return missing_image_names

# Example usage
base_directory = Path("/content/drive/MyDrive/tmas")
# Ensure that missing_images and reversed_unique_ids are properly defined
# missing_images = {'uid1': 'File does not exist', 'uid2': 'File exists', ...}

missing_image_names = copy_missing_images_and_collect_ids(base_directory, missing_images, reversed_unique_ids, file_extension="png")
print(f"Collected {len(missing_image_names)} missing image names.")


/content/drive/MyDrive/tmas/CRYPTIC/plate-images-20240423/UKMYC5/14/08-05TB12001-21742-1-14-UKMYC5-filtered.png
Copied missing image to NotAnnotate: 08-05TB12001-21742-1-14-UKMYC5-filtered.png
/content/drive/MyDrive/tmas/CRYPTIC/plate-images-20240423/UKMYC5/14/04-01059-717554-1-14-UKMYC5-filtered.png
Copied missing image to NotAnnotate: 04-01059-717554-1-14-UKMYC5-filtered.png
/content/drive/MyDrive/tmas/CRYPTIC/plate-images-20240423/UKMYC5/14/04-01106-719160-1-14-UKMYC5-filtered.png
Copied missing image to NotAnnotate: 04-01106-719160-1-14-UKMYC5-filtered.png
/content/drive/MyDrive/tmas/CRYPTIC/plate-images-20240423/UKMYC5/14/04-01075-719392-1-14-UKMYC5-filtered.png
Copied missing image to NotAnnotate: 04-01075-719392-1-14-UKMYC5-filtered.png
/content/drive/MyDrive/tmas/CRYPTIC/plate-images-20240423/UKMYC5/14/04-00522-714514-1-21-UKMYC5-filtered.png
/content/drive/MyDrive/tmas/CRYPTIC/plate-images-20240423/UKMYC5/21/04-00522-714514-1-21-UKMYC5-filtered.png
Copied missing image to NotA